In [1]:
import pandas as pd 
import numpy as np

##### I separated the scheme table and input table on excel to get two different csvs and read them in

In [2]:
input_df=pd.read_csv('Sample_Input.csv')

In [3]:
schema_df=pd.read_csv('Schema.csv')

In [4]:
print(input_df.shape)

(12751, 22)


##### Creating event time stamp column from the Date and Time columns and converting it to a datetime format

In [5]:
input_df['event_timestamp']=input_df['Date']+' ' +input_df['Time']
input_df['event_timestamp']=pd.to_datetime(input_df['event_timestamp'],format='%d/%m/%Y %H:%M:%S')
input_df.drop(['Date','Time'],axis='columns' ,inplace=True)

##### Reordering the list

In [6]:
columns=input_df.columns.values.tolist()
columns=[columns[-1]]+columns[0:-1]
input_df=input_df[columns]

##### Sampling values every 10s by max value per tag

In [7]:
input_df.set_index('event_timestamp',inplace=True)
resampled_df=input_df.resample('10s').max()
resampled_df.reset_index(inplace=True)
resampled_df.drop('Milli Sec',axis='columns',inplace=True)

In [8]:
resampled_df

,event_timestamp,L_R1SDW_SPDREF,L_R1SDD_SPDREF,L_R1SDL_SPDREF,L_R1SDW_SPDFBK,L_R1SDD_SPDFBK,L_R1SDL_SPDFBK,L_R1_ODG_AUX_B07,RM1_RUNDOWN_ID_HS,L_R1SD_WS_GAP,L_R1SD_DS_GAP,L_R1SD_WS_SPDREF,L_R1SD_DS_SPDREF,L_R1SDLS_SPDREF,L_R1SD_WS_SPDFBK,L_R1SD_DS_SPDFBK,L_R1SDLS_SPDFBK,L_R1SD_WS_CUR,L_R1SD_DS_CUR,L_R1_CNT_PASS0
0,2022-05-06 21:58:30,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.680008,194.619995,0.000000,0.000000,0,0.0000,0.000000,0,0,0,1
1,2022-05-06 21:58:40,29.866667,29.866667,0,16.7824,16.812799,0,1,225726050,194.678009,194.612000,29.866667,29.866667,0,16.7824,16.812799,0,177,213,0
2,2022-05-06 21:58:50,0.000000,0.000000,0,0.1552,0.148800,0,1,225726050,167.284012,167.197998,0.000000,0.000000,0,0.1552,0.148800,0,57,54,0
3,2022-05-06 21:59:00,14.952726,14.952726,0,10.5504,10.556801,0,1,225726050,167.266006,167.185989,14.952726,14.952726,0,10.5504,10.556801,0,187,205,0
4,2022-05-06 21:59:10,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,154.500000,154.487991,0.000000,0.000000,0,0.0000,0.000000,0,46,46,0
5,2022-05-06 21:59:20,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,192.596008,192.447998,0.000000,0.000000,0,0.0000,0.000000,0,189,172,1
6,2022-05-06 21:59:30,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.502014,194.419998,0.000000,0.000000,0,0.0000,0.000000,0,51,52,1


In [9]:
resampled_df.shape

(7, 20)

##### After this I was unsure 